In [2]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Manipulação de datas
from datetime import datetime, timedelta, date
import time

# Coleta cotações
import yfinance as yf

# Lista com os ativos
import __list_ativos__

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Lista com o nome dos ativos
acao = __list_ativos__.lst_acao2

In [7]:
# Pegando as datas para os últimos 10 dias úteis
date1, date2 = date.today() - timedelta(days=2), date.today() - timedelta(days=0)

res = np.busday_count(date1.strftime('%Y-%m-%d'),
                    date2.strftime('%Y-%m-%d'))
i = 1
while res < 30:
    test_date1 = date.today() - timedelta(days=i)
    res = np.busday_count(test_date1.strftime('%Y-%m-%d'),
                    date2.strftime('%Y-%m-%d'))
    i = i + 1

In [11]:
# Inicio e fim das data de coletas das cotações
start = test_date1 
end = date2

# Coletando as cotações de fechamento
df = pd.DataFrame()
for i in acao:
    df[i] = yf.download(f'{i}.SA', start=test_date1, end=date2, progress=False, threads=False)['Adj Close']
    df.append(df)
    time.sleep(1)

df = df.tail(20)

# Calculando os retonos diários
df = round(df.pct_change() * 100, 2)

# Apagando os valores nulos
df.dropna(inplace=True)

# Reset do index
df.reset_index(inplace=True)

# Dias
dias = df.shape[0]

In [12]:
df.head(100)

,Date,AALR3,ABCB4,ABEV3,ADHM3
0,2022-11-30,-0.59,-0.91,1.02,0.0
1,2022-12-01,0.00,-2.04,1.01,0.0
2,2022-12-02,-0.25,3.12,1.55,0.0
3,2022-12-05,-0.84,-1.77,-1.90,0.0
4,2022-12-06,-1.15,-0.51,0.19,0.0
5,2022-12-07,-0.51,0.05,-0.31,0.0
6,2022-12-08,4.38,-3.30,-1.88,0.0
7,2022-12-09,2.10,-0.43,-1.21,0.0
8,2022-12-12,-1.77,-1.45,-1.74,0.0
9,2022-12-13,-1.85,-2.34,0.26,0.0


In [ ]:
df.head()

In [ ]:
# Coletando os valores da coluna que era index (Data)
lista_date = pd.DataFrame(df['Date'])
lista_date['Date2'] = lista_date['Date'].dt.date
lista_date2 = list(lista_date['Date2'])

In [ ]:
# Apagando a coluna 'Date' do DataFrame
df.drop(['Date'], axis=1, inplace=True)

# Fazendo a transposição linhas para colunas | colunas para linhas
df = df.T

df.head()

In [ ]:
# Criando uma nova 'variável' coluna com os retornos acumulados
df['Total_Acumulado'] = df.sum(axis=1)

# Renomeando as colunas
j = 0
for i in df.columns[:dias]:
    df.rename(columns={i:f'{lista_date2[j]}'}, inplace=True)
    j = j + 1

# Ordenando pelo maior retorno
df.sort_values(by='Total_Acumulado', ascending=False)    

In [ ]:
# Criando um DataFrame só com o Total acumulado de 10 dias
df_filter = pd.DataFrame(df['Total_Acumulado'])

# Reset do Index
df_filter.reset_index(inplace=True)

# Renomeando coluna 'index' -> 'Ação
df_filter.rename(columns={'index':'Ação'}, inplace=True)

# Salva em um documento csv
df_filter

In [7]:
d = pd.read_csv('../Api/retornos/cotacoes.csv', sep=';')
d

,AALR3,ABCB4,ABEV3,ADHM3,AERI3,AESB3,AFLT3
2022-09-08,19.700001,19.358786,14.724186,1.56,2.53,9.85,10.17
2022-09-09,19.700001,18.877272,14.933174,1.56,2.49,9.81,10.50
2022-09-12,19.790001,19.457052,14.914175,1.56,2.53,9.79,10.38
2022-09-13,19.650000,19.093462,14.809681,1.56,2.43,9.80,9.97
2022-09-14,20.070000,19.928740,14.610192,1.56,2.38,9.77,9.96
...,...,...,...,...,...,...,...
2022-12-21,21.200001,19.260000,14.350000,1.56,1.00,9.56,9.12
2022-12-22,21.100000,19.450001,14.420000,1.56,1.00,9.71,9.32
2022-12-23,21.309999,19.719999,14.620000,1.56,1.04,9.96,9.32
2022-12-26,21.200001,19.540001,14.520000,1.56,1.03,9.71,9.22
